# 发送email与接收email
## 发送email
SMTP是发送邮件的协议，Python内置对SMTP的支持，可以发送纯文本邮件、HTML邮件以及带附件的邮件。
Python对SMTP支持有smtplib和email两个模块，email负责构造邮件，smtplib负责发送邮件。

首先，我们来构造一个最简单的纯文本邮件：

In [2]:
from email.mime.text import MIMEText
msg = MIMEText('hello, send by Python...', 'plain', 'utf-8')
msg

注意到构造MIMEText对象时，第一个参数就是邮件正文，第二个参数是MIME的subtype，传入'plain'表示纯文本，最终的MIME就是'text/plain'，最后一定要用utf-8编码保证多语言兼容性。

然后，通过SMTP发出去：

In [6]:
# 输入Email地址和口令:
from_addr = '469165171@qq.com'#input('From: ')
password = 'C1h5E8n5'#input('Password: ')
# 输入收件人地址:
to_addr = '469165171@qq.com'#input('To: ')
# 输入SMTP服务器地址:
smtp_server = 'smtp.qq.com'#input('SMTP server: ')

import smtplib
server = smtplib.SMTP(smtp_server, 25) # SMTP协议默认端口是25
server.set_debuglevel(1)
server.login(from_addr, password)
server.sendmail(from_addr, [to_addr], msg.as_string())
server.quit()

send: 'ehlo chenqiangdeMacBook-Pro.local\r\n'
reply: b'250-newxmesmtplogicsvrsza2-0.qq.com\r\n'
reply: b'250-PIPELINING\r\n'
reply: b'250-SIZE 73400320\r\n'
reply: b'250-STARTTLS\r\n'
reply: b'250-AUTH LOGIN PLAIN XOAUTH XOAUTH2\r\n'
reply: b'250-AUTH=LOGIN\r\n'
reply: b'250-MAILCOMPRESS\r\n'
reply: b'250 8BITMIME\r\n'
reply: retcode (250); Msg: b'newxmesmtplogicsvrsza2-0.qq.com\nPIPELINING\nSIZE 73400320\nSTARTTLS\nAUTH LOGIN PLAIN XOAUTH XOAUTH2\nAUTH=LOGIN\nMAILCOMPRESS\n8BITMIME'
send: 'AUTH PLAIN ADQ2OTE2NTE3MUBxcS5jb20AQzFoNUU4bjU=\r\n'
reply: b'535 Login Fail. Please enter your authorization code to login. More information in http://service.mail.qq.com/cgi-bin/help?subtype=1&&id=28&&no=1001256\r\n'
reply: retcode (535); Msg: b'Login Fail. Please enter your authorization code to login. More information in http://service.mail.qq.com/cgi-bin/help?subtype=1&&id=28&&no=1001256'
send: 'AUTH LOGIN NDY5MTY1MTcxQHFxLmNvbQ==\r\n'
reply: b'334 UGFzc3dvcmQ6\r\n'
reply: retcode (334); Msg: b

SMTPAuthenticationError: (535, b'Login Fail. Please enter your authorization code to login. More information in http://service.mail.qq.com/cgi-bin/help?subtype=1&&id=28&&no=1001256')